Copy `.streamlit` to `notebooks` for `st.secrets` to work.

In [1]:
import streamlit as st
from congress import Congress
from deta import Deta

In [67]:
PROPUBLICA_API_KEY = st.secrets["PROPUBLICA_API_KEY"];
DETA_API_KEY = st.secrets["DETA_API_KEY"];
DETA_ID = st.secrets["DETA_ID"];

# Chamber
SENATE = 'senate'
HOUSE = 'house'

congress = Congress(PROPUBLICA_API_KEY)

# How it works
1. Get all passed bills
2. For each passed bill, for each chamber, get the date of passage
3. For each date and chamber, call vote by date
4. For each ...

DRAFT BELOW
Rework voting record call
- Call get bills passed
- Call votes by date for the date for each chamber from the bills passed
- calls votes.get date per chamber and rollcall number, and session <- NOT NEEDED this is included with the above
- Returns list of members and how they voted
- Display what they voted on per bill


In [80]:
def get_bill_vote_data(chamber, date, bill_id):
    votes = congress.votes.by_date(chamber, date)

    filtered_vote = [vote for vote in votes['votes'] if vote['bill']['bill_id'] == bill_id]
    assert(len(filtered_vote) == 1)

    bill_vote_by_date = filtered_vote[0]

    return congress.votes.get(
        chamber      = bill_vote_by_date['chamber'],
        rollcall_num = bill_vote_by_date['roll_call'],
        session      = bill_vote_by_date['session'],
        congress     = bill_vote_by_date['congress'],
    )


def get_bill_vote_positions(passed_bill):
    house_date  = passed_bill['house_passage']
    senate_date = passed_bill['senate_passage']
    bill_id     = passed_bill['bill_id']

    print(f'Bill ID: {bill_id}, Passed House: {house_date}, Passed Senate: {senate_date}')

    # print('Getting all house votes on', house_date)

    house_votes = get_bill_vote_data(HOUSE, house_date, bill_id)
    house_vote_positions = house_votes['votes']['vote']['positions']

    senate_votes = get_bill_vote_data(SENATE, senate_date, bill_id)
    senate_vote_positions = senate_votes['votes']['vote']['positions']

    return {
        bill_id: bill_id,
        house_passage: house_date,
        senate_passage: senate_date,
        house_vote_positions: house_vote_positions,
        senate_vote_positions: senate_vote_positions
    }


congress_num = 117

passed_bills = congress.bills.passed(chamber=SENATE, congress=congress_num)
f = get_bill_vote_positions(passed_bills['bills'][0])
f

Bill ID: s5168-117, Passed House: 2022-12-21, Passed Senate: 2022-12-15


KeyError: 'bill_id'

In [78]:
f['votes'].keys()
f['votes']['vote'].keys()
f['votes']['vote']['positions']

[{'member_id': 'A000370',
  'name': 'Alma Adams',
  'party': 'D',
  'state': 'NC',
  'district': '12',
  'cook_pvi': None,
  'vote_position': 'Yes',
  'dw_nominate': -0.465},
 {'member_id': 'A000055',
  'name': 'Robert B. Aderholt',
  'party': 'R',
  'state': 'AL',
  'district': '4',
  'cook_pvi': None,
  'vote_position': 'Yes',
  'dw_nominate': 0.382},
 {'member_id': 'A000371',
  'name': 'Pete Aguilar',
  'party': 'D',
  'state': 'CA',
  'district': '31',
  'cook_pvi': None,
  'vote_position': 'Yes',
  'dw_nominate': -0.295},
 {'member_id': 'A000372',
  'name': 'Rick Allen',
  'party': 'R',
  'state': 'GA',
  'district': '12',
  'cook_pvi': None,
  'vote_position': 'Yes',
  'dw_nominate': 0.696},
 {'member_id': 'A000376',
  'name': 'Colin Allred',
  'party': 'D',
  'state': 'TX',
  'district': '32',
  'cook_pvi': None,
  'vote_position': 'Yes',
  'dw_nominate': None},
 {'member_id': 'A000369',
  'name': 'Mark Amodei',
  'party': 'R',
  'state': 'NV',
  'district': '2',
  'cook_pvi': N

In [65]:
help(congress.votes.get)

Help on method get in module congress.votes:

get(chamber, rollcall_num, session, congress=118) method of congress.votes.VotesClient instance
    Return a specific roll-call vote, including a complete list of member positions



In [60]:
f['roll_call']

533

## 1. Get all passed bills

In [26]:
congress_num = 117

passed_bills = congress.bills.passed(chamber=SENATE, congress=congress_num)
print(passed_bills.keys())

dict_keys(['congress', 'chamber', 'num_results', 'offset', 'bills'])


In [27]:
print(passed_bills['bills'][0].keys())
# passed_bills['bills'][0]['summary']

dict_keys(['bill_id', 'bill_slug', 'bill_type', 'number', 'bill_uri', 'title', 'short_title', 'sponsor_title', 'sponsor_id', 'sponsor_name', 'sponsor_state', 'sponsor_party', 'sponsor_uri', 'gpo_pdf_uri', 'congressdotgov_url', 'govtrack_url', 'introduced_date', 'active', 'last_vote', 'house_passage', 'senate_passage', 'enacted', 'vetoed', 'cosponsors', 'cosponsors_by_party', 'committees', 'committee_codes', 'subcommittee_codes', 'primary_subject', 'summary', 'summary_short', 'latest_major_action_date', 'latest_major_action'])


## 2. Get the votes on the dates of passage for House and Senate

In [28]:
passed_bills['bills'][0]['house_passage'], passed_bills['bills'][0]['senate_passage']

('2022-12-21', '2022-12-15')

In [29]:
house_votes = congress.votes.by_date(HOUSE, passed_bills['bills'][0]['house_passage'])

print(house_votes.keys())

dict_keys(['chamber', 'start_date', 'end_date', 'num_results', 'votes'])


## 3. Filter the vote for said bills out of all the votes for that day.

In [30]:
print(house_votes['votes'][0].keys())

dict_keys(['congress', 'chamber', 'session', 'roll_call', 'source', 'url', 'vote_uri', 'bill', 'amendment', 'question', 'question_text', 'description', 'vote_type', 'date', 'time', 'result', 'democratic', 'republican', 'independent', 'total'])


In [64]:
house_votes['votes'][1]['bill']

{'bill_id': 's3168-117',
 'number': 'S.3168',
 'sponsor_id': 'K000377',
 'api_uri': 'https://api.propublica.org/congress/v1/117/bills/s3168.json',
 'title': 'A bill to amend the White Mountain Apache Tribe Water Rights Quantification Act of 2010 to modify the enforceability date for certain provisions, and for other purposes.',
 'latest_action': 'Signed by President.'}

In [53]:
# [vote for vote in house_votes['votes'] if vote['bill'] == 's5168-117']
[vote['bill']['bill_id'] for vote in house_votes['votes']]


['s3308-117',
 's3168-117',
 'hr1917-117',
 's2834-117',
 's4926-117',
 's5168-117',
 's2333-117',
 'hres1529-117']

In [50]:
house_votes['votes']

[{'congress': 117,
  'chamber': 'House',
  'session': 2,
  'roll_call': 538,
  'source': 'http://clerk.house.gov/evs/2022/roll538.xml',
  'url': 'http://clerk.house.gov/evs/2022/roll538.xml',
  'vote_uri': 'https://api.propublica.org/congress/v1/117/house/sessions/2/votes/538.json',
  'bill': {'bill_id': 's3308-117',
   'number': 'S.3308',
   'sponsor_id': 'K000377',
   'api_uri': 'https://api.propublica.org/congress/v1/117/bills/s3308.json',
   'title': 'A bill to authorize the Colorado River Indian Tribes to enter into lease or exchange agreements and storage agreements relating to water of the Colorado River allocated to the Colorado River Indian Tribes, and for other purposes.',
   'latest_action': 'Presented to President.'},
  'amendment': {},
  'question': 'On Motion to Suspend the Rules and Pass',
  'question_text': '',
  'description': 'Colorado River Indian Tribes Water Resiliency Act',
  'vote_type': '2/3 YEA-AND-NAY',
  'date': '2022-12-21',
  'time': '23:26:00',
  'result':

In [19]:
print(dir(congress.votes))

['BASE_URI', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', 'apikey', 'by_date', 'by_month', 'by_range', 'by_type', 'fetch', 'get', 'http', 'loneno', 'missed', 'nominations', 'party', 'perfect', 'today']


In [21]:
help(congress.fetch)

Help on method fetch in module congress.client:

fetch(path, parse=<function Client.<lambda> at 0x7f2884939f30>) method of congress.Congress instance
    Make an API request, with authentication.
    
    This method can be used directly to fetch new endpoints or customize parsing.
    
    ::
    
        >>> from congress import Congress
        >>> client = Congress()
        >>> senate = client.fetch('115/senate/members.json')
        >>> print(senate['num_results'])
        101



## Step X: Build populate database in Deta